In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler


train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
train_new = pd.read_csv('./data/train_new.csv')
test_new = pd.read_csv('./data/test_new.csv')

train = pd.concat([train, train_new], axis=1)
test = pd.concat([test, test_new], axis=1)
train = train.fillna(-999)
test = test.fillna(-999)

features = [c for c in train.columns if c not in ["id","Y1", "Y2"]]

X = train[features].values
X_test = test[features].values
y2 = train["Y2"].values

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping

scaler = StandardScaler()
X = scaler.fit_transform(train[features].values)
X_test = scaler.transform(test[features].values)

n_features = X.shape[1]

mlp_model = Sequential([
    Input(shape=(n_features,)),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1)
])


mlp_model.compile(optimizer='adam', loss='mse')

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
hist = mlp_model.fit(X, y2, 
                     epochs=50, 
                     batch_size=64, 
                     callbacks=[early_stop], 
                     validation_split=0.2,
                     verbose=1)
y2_pred = mlp_model.predict(X_test).flatten()
y1_base_pred = np.full(len(test), train["Y1"].mean())
submission = pd.DataFrame({
    "id": test["id"],
    "Y1": y1_base_pred,
    "Y2": y2_pred
})
submission.to_csv("mlp_submission.csv", index=False)


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam



train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
train_new = pd.read_csv('./data/train_new.csv')
test_new = pd.read_csv('./data/test_new.csv')

train = pd.concat([train, train_new], axis=1)
test = pd.concat([test, test_new], axis=1)

train = train.fillna(-999)
test = test.fillna(-999)

features = [c for c in train.columns if c not in ["id","Y1", "Y2"]]
X_full = train[features].values
X_test = test[features].values
y2_full = train["Y2"].values

def make_model(inut_dim):
    model = Sequential()
    model.add(Input(shape=(inut_dim,)))
    model.add(BatchNormalization())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof = np.zeros(len(train))
test_pred = np.zeros((len(test), kf.get_n_splits()))
fold_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_full),1):
    X_tr, X_va = X_full[train_idx], X_full[val_idx]
    y_tr, y_va = y2_full[train_idx], y2_full[val_idx]

    x_scaler = StandardScaler().fit(X_tr)
    X_tr_s = x_scaler.transform(X_tr)
    X_va_s = x_scaler.transform(X_va)
    X_tst_s = x_scaler.transform(X_test)

    callbacks = [ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1),EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True, verbose=1)]
    model = make_model(X_tr_s.shape[1])
    history = model.fit(X_tr_s, y_tr, 
                        validation_data=(X_va_s, y_va),
                        epochs=200, 
                        batch_size=128,
                        callbacks=callbacks,
                        verbose=1)
    
    va_pred = model.predict(X_va_s, verbose=0).ravel()
    oof[val_idx] = va_pred

    fold_r2 = r2_score(y_va, va_pred)
    fold_scores.append(fold_r2)
    print(f"Fold {fold} R2: {fold_r2}")

    te_pred =  model.predict(X_tst_s, verbose=0).ravel()
    test_pred[:, fold-1] = te_pred

oof_r2 = r2_score(y2_full, oof)
print(f"OOF R2: {oof_r2}")
print("Test R2 scores by fold:", fold_scores)
print(f"Mean Fold R2: {np.mean(fold_scores)} +/- {np.std(fold_scores)}")

y2_test_pred = test_pred.mean(axis=1)

submission = pd.DataFrame({
    "id": test["id"],
    "Y1": np.full(len(test), train["Y1"].mean()),
    "Y2": y2_test_pred
})
submission.to_csv("mlp_kfold_submission.csv", index=False)



Epoch 1/200
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.3729 - val_loss: 0.2837 - learning_rate: 0.0010
Epoch 2/200
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3360 - val_loss: 0.2891 - learning_rate: 0.0010
Epoch 3/200
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3098 - val_loss: 0.3195 - learning_rate: 0.0010
Epoch 4/200
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.2921 - val_loss: 0.2662 - learning_rate: 0.0010
Epoch 5/200
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2947 - val_loss: 0.2853 - learning_rate: 0.0010
Epoch 6/200
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.2846 - val_loss: 0.2841 - learning_rate: 0.0010
Epoch 7/200
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.2825 - val_loss: 0.2814 - learning_rate: 0.0010
Epoch 8/200
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.2851 - val_loss: 0.2893 - learning_rate: 0.0010
Epoch 9/200
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.2794 - val_loss: 0.2661 - learning_rate: 0.0010
E

In [6]:
final = "final.csv"
y2file = "mlp_kfold_submission.csv"

final_df = pd.read_csv(final)
y2_df = pd.read_csv(y2file)

y2_values = y2_df[['id',"Y2"]]
updated_df = pd.merge(final_df.drop('Y2', axis=1), y2_values, on='id', how='left')
updated_df.to_csv("final_updated.csv", index=False)